# Cross Tokenizer Knowledge Distillation
- Quelle: https://github.com/Nicolas-BZRD/llm-recipes

## Activate Environment

In [1]:
conda activate llm-distillation

(/home/thsch026/my-envs/llm-distillation) 


## 1. Im ersten Schritt wird aus einem Datenset eine Antwort Datei mit dem Teacher Modell erstellt
- Für "train" und "validation" getrennt ausführen

### Erstellt die Antwort Datei für die Distillation mit Mistral-7B-Instruct-v0.2 als teacher - Basis Dataset ist rajpurkar/squad

- Muss einmal mit train und einmal mit validation ausgeführt werden

In [ ]:
cd /home/thsch026/

python llm-distillation/datasets/generator.py \
--model_id mistralai/Mistral-7B-Instruct-v0.2 \
--model_tokenizer mistralai/Mistral-7B-Instruct-v0.2 \
--dataset_id rajpurkar/squad \
--split_name train \
--number_few_shot 5 \
--batch_size 4 \
--bfloat \
--task qa_generative \
--mapping llm-distillation/benchmark/mapping/squad.json

### Erstellt die Antwort Datei für die Distillation mit Llama-2-7b-chat-hf als teacher - Basis Dataset ist rajpurkar/squad
- Muss einmal mit train und einmal mit validation ausgeführt werden

In [ ]:
cd /home/thsch026/

python llm-distillation/datasets/generator.py \
--model_id meta-llama/Llama-2-7b-chat-hf \
--model_tokenizer meta-llama/Llama-2-7b-chat-hf \
--dataset_id rajpurkar/squad \
--split_name validation \
--number_few_shot 5 \
--batch_size 8 \
--task qa_generative \
--mapping llm-distillation/benchmark/mapping/squad.json

## 2. Im zweiten Schritt wird die eigentliche Knowledge Distillation mit dem Student Model durchgeführt 
- Vorher müssen die Antwortdateien (für train und vaildation) mit dem jeweiligen Datenset und dem Lehremodell erstellt werden - Schritt 1


In [2]:
export CUDA_VISIBLE_DEVICES=2

(/home/thsch026/my-envs/llm-distillation) 


In [ ]:
cd /home/thsch026/
export SAVEDIR="/home/thsch026/masterarbeit/models/generated/dist/Llama2_dist_pythia-410_deduped"

python -m torch.distributed.launch llm-recipes/finetuning.py \
--model_name EleutherAI/pythia-410m-deduped \
--dataset.file llm-distillation/datasets/loader/squad.py \
--lr 1e-6 \
--num_epochs 1 \
--run_validation True \
--batch_size_training 8 \
--val_batch_size 8 \
--save_step 10 \
--output_dir train/distill \
--distillation_config.model_name meta-llama/Llama-2-7b-chat-hf \
--distillation \
--distillation_config.enable_fsdp \
--distillation_config.distil_factor 1.5 


(/home/thsch026/my-envs/llm-distillation) 
(/home/thsch026/my-envs/llm-distillation) 
(/home/thsch026/my-envs/llm-distillation) 
/home/thsch026/my-envs/llm-distillation/lib/python3.9/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Clearing GPU cache for all ranks
--> Model EleutherAI/pythia-410m-deduped

--> EleutherAI/pythia-410m-deduped has 405.334016 Million params

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:10<00:00,  5.18s/it]
--> Model meta-llama/Llama-2-7b-chat-hf

--> meta-llama/Llama-2-7b-chat-hf has 6738.415616 Million params

/home/thsch026/my-envs/llm-distillation/lib

###  3. Ein Beispiel für den Versuch das ganze mit Mistral und dem squad datenset zu machen
- Auch wenn run_validation auf False steht muss ein validation Dataset vorhanden sein!
- Das Loader File ist recht knifflig. Es soll beide Datensätze laden, validation und train
- Epochs habe ich auf 1 gesetzt, wobei das schon lange dauert

In [2]:
export CUDA_VISIBLE_DEVICES=2

(/home/thsch026/my-envs/llm-distillation) 


In [ ]:
cd /home/thsch026/
export SAVEDIR="/home/thsch026/masterarbeit/models/generated/dist/Llama2_dist_pythia-410_deduped"

python -m torch.distributed.launch llm-recipes/finetuning.py \
--model_name EleutherAI/pythia-410m-deduped \
--enable_fsdp \
--run_validation False \
--dataset.file llm-distillation/datasets/loader/squad.py \
--lr 1e-6 \
--num_epochs 1 \
--batch_size_training 4 \
--val_batch_size 4 \
--output_dir $SAVEDIR \
--save_step 1000 \
--distillation \
--distillation_config.model_name mistralai/Mistral-7B-Instruct-v0.2 \
--distillation_config.enable_fsdp \
--distillation_config.distil_factor 1.5

#### Variablen setzen als Workaround (Wird wohl nicht gebraucht)

In [3]:
export RANK=1
export WORLD_SIZE=1
export MASTER_ADDR=localhost
export MASTER_PORT=12345
cd /home/thsch026/masterarbeit/experiment

(/home/thsch026/my-envs/torchtune) 
(/home/thsch026/my-envs/torchtune) 
(/home/thsch026/my-envs/torchtune) 
(/home/thsch026/my-envs/torchtune) 
(/home/thsch026/my-envs/torchtune) 


Hier ein funktionierender Approach für das squad datenset